In [258]:
from IPython.display import display, display_html
from IPython.display import HTML
import IPython.core.display as di

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

Toggle code

In [254]:
!!pip install urllib3

['Requirement already satisfied: urllib3 in ./.venv/lib/python3.8/site-packages (1.26.3)',
 'WARNING: You are using pip version 20.2.3; however, version 21.0.1 is available.',
 "You should consider upgrading via the '/private/tmp/IPrecon/.venv/bin/python3 -m pip install --upgrade pip' command."]

In [264]:
SHODAN_APIKEY = ""
SCAMALYTICS_APIKEY = ""
ABUSEIPDB_APIKEY = ""

In [260]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [261]:
from IPython.core.display import display, display_html, HTML
from IPython.display import Javascript, display
import ipywidgets as widgets

IPwidget=widgets.Text(
    value='',
    placeholder='<IP Address>',
    description='IP Address:',
    disabled=False
)
display(IPwidget)

def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))

button = widgets.Button(description="Run")
button.on_click(run_all)
display(button)

Text(value='', description='IP Address:', placeholder='<IP Address>')

Button(description='Run', style=ButtonStyle())

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [279]:
from urllib.request import urlopen
from urllib.request import Request
import urllib3
import json
import socket
import time


http = urllib3.PoolManager()

INPUT_IP=IPwidget.value

def run_scamalytics():
  url = f"https://api12.scamalytics.com/netscylla?ip={INPUT_IP}&key={SCAMALYTICS_APIKEY}"
  r = http.request(
      'GET',
      url
  )
  
  data=json.loads(r.data.decode('utf-8'))
  if(data['risk']=="low"):
      row_col="green"
  if(data['risk']=="medium"):
      row_col="orange"
  if(data['risk']=="high"):   
      row_col="red"
  if(data['risk']=="very high"):   
      row_col="purple"
  display(HTML("<h1>ActionFraud Score</h1>"))
  display(HTML("<table>"))
  display(HTML("<tr bgcolor="+row_col+"><td><p style=\"color:white\">Risk</p></td><td><p style=\"color:white\">"+str(data['risk'])+"</p></td><td><p style=\"color:white\">Fraud Score</p></td><td><p style=\"color:white\">"+str(data['score'])+"</p></td></tr>"))
  display(HTML("</table>"))
    
def run_shodan():
  url = f"https://api.shodan.io/shodan/host/{INPUT_IP}?key={SHODAN_APIKEY}"
  r = http.request(
      'GET',
      url
  )

  display(HTML("<h1>Shodan</h1>"))

  data=json.loads(r.data.decode('utf-8'))
  if (len(data) < 25):
    display(HTML("<p>No Data Found!</p>"))
  else:
    display(HTML("<table>"))
    display(HTML("<tr><td>ASN</td><td>"+str(data['asn'])+"</td></tr>"))
    display(HTML("<tr><td>IP</td><td>"+str(data['ip_str'])+"</td></tr>"))
    display(HTML("<tr><td>ISP</td><td>"+str(data['isp'])+"</td></tr>"))
    display(HTML("<tr><td>City</td><td>"+data['city']+", "+data['region_code']+", "+data['country_code']+"</td></tr>"))
    display(HTML("<tr><td>Long+Lat</td><td>"+str(data['longitude'])+", "+str(data['latitude'])+"</td></tr>"))
    display(HTML("<tr><td>IP</td><td>"+str(data['country_name'])+"</td></tr>"))
    display(HTML("<tr><td>Hostnames</td><td>"+str(data['hostnames'])+"</td></tr>"))
    display(HTML("<tr><td>TCP Ports</td><td>"+str(data['ports'])+"</td></tr>"))
    for i in range(len(data['ports'])):
      display(HTML("<tr><td>"+str(data['data'][i]['port'])+"</td><td>"+data['data'][i]['product']+"/"+str(data['data'][i]['version'])+"</td></tr>"))
    display(HTML("</table>"))

def run_urlhaus(): 
  display(HTML("<h1>URLHaus</h1>"))  
  #https://urlhaus-api.abuse.ch/v1/host/
  url = "https://urlhaus-api.abuse.ch/v1/host/"
  r = http.request(
      'POST',
      url,
      fields={'host': INPUT_IP} 
  )
  data=json.loads(r.data.decode('utf-8'))
  if (len(data) < 25):
    display(HTML("<p>No Data Found!</p>"))
  else:
    display(HTML("<table>"))
    display(HTML("<tr><td>Firstseen</td><td>"+str(data['firstseen'])+"</td></tr>"))
    for i in range(len(data['urls'])):  
      display(HTML("<tr><td>Urlhaus_reference</td><td>"+str(data['urls'][i]['urlhaus_reference'])+"</td></tr>"))
      display(HTML("<tr><td>URL</td><td>"+str(data['urls'][i]['url'])+"</td></tr>"))
      display(HTML("<tr><td>Status</td><td>"+str(data['urls'][i]['url_status'])+"</td></tr>")) 
      display(HTML("<tr><td>Threat</td><td>"+str(data['urls'][i]['threat'])+"</td></tr>"))
      display(HTML("<tr><td>Tags</td><td>"+str(data['urls'][i]['tags'])+"</td></tr>"))  
    display(HTML("</table>"))

def run_pulsedive():
  #pulsedive  
  display(HTML("<h1>Pulsedive</h1>"))  
  #https://urlhaus-api.abuse.ch/v1/host/
  url = "https://pulsedive.com/api/analyze.php"
  r = http.request(
      'POST',
      url,
      fields={'value': INPUT_IP, 'probe':1, 'pretty' : 1} 
  )
  data=json.loads(r.data.decode('utf-8'))
  data2="processing"
  while(data2=="processing"): 
    print("Response not ready. Sleeping for 3 seconds..")
    time.sleep(3)
    r = http.request(
      'POST',
      "https://pulsedive.com/api/analyze.php?qid="+str(data['qid'])+"&pretty=1")
    data=json.loads(r.data.decode('utf-8'))  
    data2=data['status']
  #print(data)

  display(HTML("<table>"))
  try:  
    display(HTML("<tr><td>umbrella_rank</td><td>"+str(data['data']['umbrella_rank'])+"</td></tr>"))
    display(HTML("<tr><td>umbrella_domain</td><td>"+str(data['data']['umbrella_domain'])+"</td></tr>"))
  except:
    display(HTML("<tr><td>umbrella</td><td>No umbrella data!</td></tr>"))
  try:  
    for i in range(len(data['data']['attributes']['port'])):
      display(HTML("<tr><td>Port</td><td>"+str(data['data']['attributes']['port'][i])+"/"+str(data['data']['attributes']['protocol'][i])+"</td></tr>"))
  except:
    display(HTML("<tr><td>Port</td><td>No port data!</td></tr>"))
  try:  
    for i in range(len(data['data']['properties']['dns'])):
      display(HTML("<tr><td>DNS PTR</td><td>"+str(data['data']['properties']['dns']['PTR'])+"</td></tr>"))
  except:
    display(HTML("<tr><td>DNS</td><td>No DNS data!</td></tr>"))
  try:
    display(HTML("<tr><td>City</td><td>"+str(data['data']['properties']['geo']['city'])+","+str(data['data']['properties']['geo']['country'])+"</td></tr>"))
  except:
    display(HTML("<tr><td>DNS</td><td>No City data!</td></tr>"))
  display(HTML("<tr><td>ISP</td><td>"+str(data['data']['properties']['geo']['isp'])+"</td></tr>"))
  display(HTML("<tr><td>ASN</td><td>"+str(data['data']['properties']['geo']['asn'])+"</td></tr>"))
  display(HTML("</table>"))

def run_abuseipdb():
  url = f"https://api.abuseipdb.com/api/v2/check?ipAddress={INPUT_IP}"
  r = http.request(
      'GET',
      url,
      headers={'Key': ABUSEIPDB_APIKEY} 
  )

  ipdb_rating=""
  data=json.loads(r.data.decode('utf-8')) 
  if(data['data']['abuseConfidenceScore']<25):
      row_col="green"
      ipdb_rating="Low"
  if(data['data']['abuseConfidenceScore']>=25 and data['data']['abuseConfidenceScore']<70):
      row_col="orange"
      ipdb_rating="Medium"
  if(data['data']['abuseConfidenceScore']>=70):   
      row_col="red"
      ipdb_rating="High"
  display(HTML("<h1>AbuseIPDB Score</h1>"))
  display(HTML("<table>"))
  display(HTML("<tr bgcolor="+row_col+"><td><p style=\"color:white\">Risk</p></td><td><p style=\"color:white\">"+ipdb_rating+"</p></td><td><p style=\"color:white\">Score</p></td><td><p style=\"color:white\">"+str(data['data']['abuseConfidenceScore'])+"</p></td></tr>"))
  display(HTML("</table>"))   
    
try:
  socket.inet_aton(INPUT_IP)
  if (SCAMALYTICS_APIKEY != ""):  
    run_scamalytics()
  if (ABUSEIPDB_APIKEY != ""):
    run_abuseipdb()
  if (SHODAN_APIKEY != ""):
    run_shodan()
    
  run_urlhaus()
  run_pulsedive()


except socket.error:
  display(HTML("Not a valid IP"))


b'{"status":"ok","mode":"live","ip":"195.58.38.73","score":"100","risk":"very high","url":"https:\\/\\/scamalytics.com\\/ip\\/195.58.38.73","credits":{"used":87,"remaining":4913},"exec":"3.08 ms"}'


Risk,very high,Fraud Score,100


Risk,Low,Score,0


Response not ready. Sleeping for 3 seconds..


umbrella_rank,None


...

Port,80/FTP


Port,22/SSH


City,"Düsseldorf,Germany"


ISP,myLoc managed IT AG


ASN,AS24961
